In [47]:
!pip install zeep

Defaulting to user installation because normal site-packages is not writeable


In [48]:
from zeep import Client
from zeep.wsse.username import UsernameToken
from base64 import b64decode
from zipfile import ZipFile
import pandas as pd
import numpy as np
import csv
import io
import json
import os, time, datetime
from datetime import datetime
from time import perf_counter 

In [49]:
dict_creden_TERYT = {'username' : 'TestPubliczny',
                    'password' : '1234abcd',
                    'wsdl' : 'https://uslugaterytws1test.stat.gov.pl/wsdl/terytws1.wsdl'}

In [50]:
token = UsernameToken(
    username=dict_creden_TERYT['username'],
    password=dict_creden_TERYT['password']
)

In [51]:
client = Client(wsdl=dict_creden_TERYT['wsdl'], wsse=token)
STATE_DATE = datetime.now()
catalog = client.service.PobierzKatalogTERC(STATE_DATE)
filename = catalog['nazwa_pliku']
content = catalog['plik_zawartosc']
decoded = b64decode(content)

In [52]:
# with open('../../../DATA/DATA_OUT/TERYT/'+filename, 'wb') as file:
with open(filename, 'wb') as file:
    file.write(decoded)
    file.close()

In [53]:
## sprawdzamy w katalogu czy jest plik TERC_Urzedowy_2023-01-26 (data dzisiejsza)

In [54]:
zf = ZipFile(filename, 'r')
# zf = ZipFile('../../../DATA/DATA_OUT/TERYT/'+filename, 'r')

In [55]:
with zf.open(zf.namelist()[1]) as csv_file:
    data = pd.read_csv(csv_file, 
                        sep=';', 
                        quoting=csv.QUOTE_NONE, 
                        encoding='utf-8',
                        dtype={
                            'WOJ' : str,
                            'POW' : str, 
                            'GMI' : str, 
                            'RODZ' : str, 
                            'NAZWA' : str, 
                            'STAN_NA'  : str       
                         })

In [56]:
data

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,NaN,NaN,NaN,DOLNOŚLĄSKIE,województwo,2023-01-01
1,02,01,NaN,NaN,bolesławiecki,powiat,2023-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2023-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2023-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2023-01-01
...,...,...,...,...,...,...,...
4259,32,61,01,1,Koszalin,gmina miejska,2023-01-01
4260,32,62,NaN,NaN,Szczecin,miasto na prawach powiatu,2023-01-01
4261,32,62,01,1,Szczecin,gmina miejska,2023-01-01
4262,32,63,NaN,NaN,Świnoujście,miasto na prawach powiatu,2023-01-01


In [57]:
## ---------------------------------------------------------------------------------
## ------- WORK WITH DATA


data_terc = data.replace(np.nan, '', regex=True)

script_name = 'terc_voivodeship'
script_time_start = perf_counter()
script_datetime_start = datetime.now()
src_data_name = 'API_PobierzKatalogTERC'
table_name_out = 'W_TERC_VOIVODESHIP'
comment = ''

In [58]:
data_terc

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,,,,DOLNOŚLĄSKIE,województwo,2023-01-01
1,02,01,,,bolesławiecki,powiat,2023-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2023-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2023-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2023-01-01
...,...,...,...,...,...,...,...
4259,32,61,01,1,Koszalin,gmina miejska,2023-01-01
4260,32,62,,,Szczecin,miasto na prawach powiatu,2023-01-01
4261,32,62,01,1,Szczecin,gmina miejska,2023-01-01
4262,32,63,,,Świnoujście,miasto na prawach powiatu,2023-01-01


In [59]:
######## WOJ
voivodeship_df = data_terc[data_terc.NAZWA_DOD == "województwo"].reset_index(drop=True).drop([
                                                                                     "POW",
                                                                                     "GMI", 
                                                                                     "RODZ", 
                                                                                     "NAZWA_DOD"], axis=1)

In [60]:
voivodeship_df

,WOJ,NAZWA,STAN_NA
0,02,DOLNOŚLĄSKIE,2023-01-01
1,04,KUJAWSKO-POMORSKIE,2023-01-01
2,06,LUBELSKIE,2023-01-01
3,08,LUBUSKIE,2023-01-01
4,10,ŁÓDZKIE,2023-01-01
5,12,MAŁOPOLSKIE,2023-01-01
6,14,MAZOWIECKIE,2023-01-01
7,16,OPOLSKIE,2023-01-01
8,18,PODKARPACKIE,2023-01-01
9,20,PODLASKIE,2023-01-01


In [61]:
for i in np.arange(0, len(voivodeship_df.columns)):
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].astype(str)
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].replace('  ', ' ')
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].replace('\t', ' ')    
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].str.strip()
    voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].str.upper()

In [62]:

voivodeship_df = voivodeship_df.rename(columns={ 
                                                    'WOJ' : 'VOIVODESHIP_CODE', 
                                                    'NAZWA' : 'VOIVODESHIP_NAME',
                                                    'STAN_NA' : 'STATUS_DATE'})

In [63]:
voivodeship_df

,VOIVODESHIP_CODE,VOIVODESHIP_NAME,STATUS_DATE
0,02,DOLNOŚLĄSKIE,2023-01-01
1,04,KUJAWSKO-POMORSKIE,2023-01-01
2,06,LUBELSKIE,2023-01-01
3,08,LUBUSKIE,2023-01-01
4,10,ŁÓDZKIE,2023-01-01
5,12,MAŁOPOLSKIE,2023-01-01
6,14,MAZOWIECKIE,2023-01-01
7,16,OPOLSKIE,2023-01-01
8,18,PODKARPACKIE,2023-01-01
9,20,PODLASKIE,2023-01-01


In [64]:
voivodeship_df = voivodeship_df.sort_values(by=['VOIVODESHIP_CODE'])
voivodeship_df['VOIVODESHIP_ID'] = np.arange(0, len(voivodeship_df))
voivodeship_df['VOIVODESHIP_ID'] = voivodeship_df['VOIVODESHIP_ID'].astype(str)

In [65]:
voivodeship_df

,VOIVODESHIP_CODE,VOIVODESHIP_NAME,STATUS_DATE,VOIVODESHIP_ID
0,02,DOLNOŚLĄSKIE,2023-01-01,0
1,04,KUJAWSKO-POMORSKIE,2023-01-01,1
2,06,LUBELSKIE,2023-01-01,2
3,08,LUBUSKIE,2023-01-01,3
4,10,ŁÓDZKIE,2023-01-01,4
5,12,MAŁOPOLSKIE,2023-01-01,5
6,14,MAZOWIECKIE,2023-01-01,6
7,16,OPOLSKIE,2023-01-01,7
8,18,PODKARPACKIE,2023-01-01,8
9,20,PODLASKIE,2023-01-01,9


In [66]:
cols = ['VOIVODESHIP_ID',
        'VOIVODESHIP_CODE', 
        'VOIVODESHIP_NAME',
        'STATUS_DATE'
        ]

In [67]:
cols

['VOIVODESHIP_ID', 'VOIVODESHIP_CODE', 'VOIVODESHIP_NAME', 'STATUS_DATE']

In [68]:
voivodeship_df = voivodeship_df[cols]
voivodeship_df = voivodeship_df.where(pd.notnull(voivodeship_df), None)

In [69]:
voivodeship_df

,VOIVODESHIP_ID,VOIVODESHIP_CODE,VOIVODESHIP_NAME,STATUS_DATE
0,0,02,DOLNOŚLĄSKIE,2023-01-01
1,1,04,KUJAWSKO-POMORSKIE,2023-01-01
2,2,06,LUBELSKIE,2023-01-01
3,3,08,LUBUSKIE,2023-01-01
4,4,10,ŁÓDZKIE,2023-01-01
5,5,12,MAŁOPOLSKIE,2023-01-01
6,6,14,MAZOWIECKIE,2023-01-01
7,7,16,OPOLSKIE,2023-01-01
8,8,18,PODKARPACKIE,2023-01-01
9,9,20,PODLASKIE,2023-01-01


In [70]:
comment = 'Load: ' + str(len(voivodeship_df)) + ' rows'

In [71]:
voivodeship_df

,VOIVODESHIP_ID,VOIVODESHIP_CODE,VOIVODESHIP_NAME,STATUS_DATE
0,0,02,DOLNOŚLĄSKIE,2023-01-01
1,1,04,KUJAWSKO-POMORSKIE,2023-01-01
2,2,06,LUBELSKIE,2023-01-01
3,3,08,LUBUSKIE,2023-01-01
4,4,10,ŁÓDZKIE,2023-01-01
5,5,12,MAŁOPOLSKIE,2023-01-01
6,6,14,MAZOWIECKIE,2023-01-01
7,7,16,OPOLSKIE,2023-01-01
8,8,18,PODKARPACKIE,2023-01-01
9,9,20,PODLASKIE,2023-01-01


In [72]:
## ---------------------------------------------------------------------------------
## ------- LOG AND INFORMATION FINISH SCRIPT

audit_list_key = []
script_time_finish = perf_counter()
script_datetime_finish = datetime.now()
script_time = script_time_finish - script_time_start
print(script_name + ' duration time: ', script_time)
print(script_name + ' start time: ', script_datetime_start)
print(script_name + ' end time: ', script_datetime_finish)


        
        
log_df = pd.DataFrame({'SCRIPT_NAME' : [script_name],
                    'SCRIPT_DATETIME_START' : [script_datetime_start],
                    'SCRIPT_DATETIME_FINISH' : [script_datetime_finish],
                    'SCRIPT_TIME' : [script_time],
                    'SRC_DATA_NAME' : [src_data_name],
                    'TABLE_NAME_OUT' : [table_name_out],
                    'COMMENT' : [comment]
                     })

terc_voivodeship duration time:  0.23501400000009198
terc_voivodeship start time:  2023-01-28 11:16:42.881919
terc_voivodeship end time:  2023-01-28 11:16:43.116467


,SCRIPT_NAME,SCRIPT_DATETIME_START,SCRIPT_DATETIME_FINISH,SCRIPT_TIME,SRC_DATA_NAME,TABLE_NAME_OUT,COMMENT
0,terc_voivodeship,2023-01-28 11:16:42.881919,2023-01-28 11:16:43.116467,0.235014,API_PobierzKatalogTERC,W_TERC_VOIVODESHIP,Load: 16 rows
